# Preference Alignment with Direct Preference Optimization (DPO)

This notebook will guide you through the process of fine-tuning a language model using Direct Preference Optimization (DPO). We will use the SmolLM2-135M-Instruct model which has already been through a SFT training, so it it compatible with DPO. You can also use the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
     <h2 style='margin: 0;color:blue'>Exercise: Aligning SmolLM2 with DPOTrainer</h2>
     <p>Take a dataset from the Hugging Face hub and align a model on it. </p> 
     <p><b>Difficulty Levels</b></p>
     <p>🐢 Use the `trl-lib/ultrafeedback_binarized` dataset</p>
     <p>🐕 Try out the `argilla/ultrafeedback-binarized-preferences` dataset</p>
     <p>🦁 Select a dataset that relates to a real-world use case you’re interested in, or use the model you trained in 
        <a href="../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb">1_instruction_tuning</a></p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login

login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

## Import libraries


In [2]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig

## Format dataset

In [3]:
# Load dataset

# TODO: 🦁🐕 change the dataset to one of your choosing
dataset = load_dataset(path="trl-lib/ultrafeedback_binarized", split="train")

In [4]:
from IPython.display import display, HTML

display(
    HTML(
        """<iframe
  src="https://huggingface.co/datasets/trl-lib/ultrafeedback_binarized/embed/viewer/all/train?row=0"
  frameborder="0"
  width="100%"
  height="360px"
></iframe>
"""
    )
)

In [5]:
# TODO: 🐕 If your dataset is not represented as conversation lists, you can use the `process_dataset` function to convert it.

def process_dataset(samples) -> dict[str, str]:
    return {
        "prompt": ["Question: " + samples["chosen"][i][0]["content"] + "\n\nAnswer: " for i in range(len(samples["chosen"]))],
        "chosen": [samples["chosen"][i][1]["content"] for i in range(len(samples["chosen"]))],
        "rejected": [samples["rejected"][i][1]["content"] for i in range(len(samples["rejected"]))]
    }

original_columns = dataset.column_names

dataset = dataset.map(process_dataset, batched=True, remove_columns=original_columns)

print(dataset)

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 62135
})


In [20]:
train_sample = dataset[0]["chosen"]
print(train_sample)

Sure, I'd be happy to help you write a version of the classic game Snake using the pygame library! Here's a basic outline of how we can approach this:

1. First, we'll need to set up the game display and create a game object that we can use to handle the game's state.
2. Next, we'll create the game's grid, which will be used to represent the game board. We'll need to define the size of the grid and the spaces within it.
3. After that, we'll create the snake object, which will be used to represent the player's movement. We'll need to define the size of the snake and the speed at which it moves.
4. We'll also need to create a food object, which will be used to represent the food that the player must collect to score points. We'll need to define the location of the food and the speed at which it moves.
5. Once we have these objects set up, we can start handling the game's logic. We'll need to define the rules for the player's movements, such as how the player can move the snake and how th

## Select the model

We will use the SmolLM2-135M-Instruct model which has already been through a SFT training, so it it compatible with DPO. You can also use the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).


<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; width:80%; color:black'>
     <p>🦁 change the model to the path or repo id of the model you trained in <a href="../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb">1_instruction_tuning</a></p>
</div>


In [8]:
torch.cuda.is_available()

True

In [9]:
# TODO: 🦁 change the model to the path or repo id of the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb)

model_name = "HuggingFaceTB/SmolLM2-135M"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.float32,
).to(device)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-DPO"
finetune_tags = ["smol-course", "module_1"]

## Train model with DPO

In [11]:
# Training arguments
training_args = DPOConfig(
    # Training batch size per GPU
    per_device_train_batch_size=4,
    # Number of updates steps to accumulate before performing a backward/update pass
    # Effective batch size = per_device_train_batch_size * gradient_accumulation_steps
    gradient_accumulation_steps=4,
    # Saves memory by not storing activations during forward pass
    # Instead recomputes them during backward pass
    gradient_checkpointing=True,
    # Base learning rate for training
    learning_rate=5e-5,
    # Learning rate schedule - 'cosine' gradually decreases LR following cosine curve
    lr_scheduler_type="cosine",
    # Total number of training steps
    max_steps=200,
    # Disables model checkpointing during training
    save_strategy="no",
    # How often to log training metrics
    logging_steps=1,
    # Directory to save model outputs
    output_dir="smol_dpo_output",
    # Number of steps for learning rate warmup
    warmup_steps=100,
    # Use bfloat16 precision for faster training
    bf16=True,
    # Disable wandb/tensorboard logging
    report_to=None,
    # Keep all columns in dataset even if not used
    remove_unused_columns=False,
    # Enable MPS (Metal Performance Shaders) for Mac devices
    use_mps_device=device == "mps",
    # Model ID for HuggingFace Hub uploads
    hub_model_id=finetune_name,
)

In [12]:
trainer = DPOTrainer(
    # The model to be trained
    model=model,
    # Training configuration from above
    args=training_args,
    # Dataset containing preferred/rejected response pairs
    train_dataset=dataset,
    # Tokenizer for processing inputs
    processing_class=tokenizer,
    # DPO-specific temperature parameter that controls the strength of the preference model
    # Lower values (like 0.1) make the model more conservative in following preferences
    #beta=0.1,
    # Maximum length of the input prompt in tokens
    max_prompt_length=1024,
    # Maximum combined length of prompt + response in tokens
    max_length=1536,
)

E:\Curso Huggingface\smol-course\.venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_prompt_length, max_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in DPOTrainer, please use the DPOConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
E:\Curso Huggingface\smol-course\.venv\Lib\site-packages\trl\trainer\dpo_trainer.py:469: UserWarning: You passed `max_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
E:\Curso Huggingface\smol-course\.venv\Lib\site-packages\trl\trainer\dpo_trainer.py:475: UserWarning: You passed `max_prompt_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [13]:
# Train the model
trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.702800
4,0.691100
5,0.683100
6,0.699100
7,0.689100
8,0.702900
9,0.687700
10,0.705900


TrainOutput(global_step=200, training_loss=0.6588983817398548, metrics={'train_runtime': 363.5443, 'train_samples_per_second': 8.802, 'train_steps_per_second': 0.55, 'total_flos': 0.0, 'train_loss': 0.6588983817398548, 'epoch': 0.05149993562508047})

In [14]:
# Save the model
trainer.save_model(f"./{finetune_name}")

In [22]:
dpo_model_path = "./SmolLM2-FT-DPO/"
dpo_model = AutoModelForCausalLM.from_pretrained(dpo_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(dpo_model_path)

chat_template = "{content}"

# Let's test the base model before training
prompt = "Use the pygame library to write a version of the classic game Snake, with a unique twist"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = prompt

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

output = dpo_model.generate(
    inputs["input_ids"],
    max_new_tokens=500, 
    do_sample=True, 
    top_k=50,
    top_p=0.9
)

response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the pygame library to write a version of the classic game Snake, with a unique twist: you control the direction the snake is going based on the X and Y coordinates.


  • Python 3.5+
  • pygame

A simple Snake game using Python, pygame


The program requires no additional modules or libraries. In addition, the original game Snake with a twist.

You can either compile it with your Python compiler (Python3) or PyGame. If you want a more complete implementation, the game requires a few tweaks to your game engine. These are all very minor and not difficult to fix.

You can either compile it with your Python compiler (Python3), PyGame or make sure to include all necessary modules and libraries in your project:

  • make use of the sys.path or the path variable
  • use the ctypes module, to access all the functions, structs and classes
  • compile it without including any modules.

There are two ways to compile it with PyGame:

Compiling it with PyGame:


Now, you need to include all nec

In [23]:
# Save to the huggingface hub if login (HF_TOKEN is set)
trainer.push_to_hub(tags=finetune_tags)

training_args.bin:   0%|          | 0.00/6.07k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jucanbe/SmolLM2-FT-DPO/commit/fbda8d99fea4cfaa73c6feabae4cb983dabc60d7', commit_message='End of training', commit_description='', oid='fbda8d99fea4cfaa73c6feabae4cb983dabc60d7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jucanbe/SmolLM2-FT-DPO', endpoint='https://huggingface.co', repo_type='model', repo_id='jucanbe/SmolLM2-FT-DPO'), pr_revision=None, pr_num=None)

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `DPOTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.